In [330]:
import random

# 1. Población inicial
# Generar la población
def generate_individual(n):
    """Crea un individuo (una permutación de filas para cada columna)."""
    individual = list(range(n))
    random.shuffle(individual)
    return individual

def generate_population(size, n):
    """Genera una población inicial aleatoria."""
    return [generate_individual(n) for _ in range(size)]

def generate_population_with_seed(size, n, initial_individuals=None):
    """
    Genera una población inicial con individuos dados por el usuario.
    
    - size: tamaño total de la población
    - n: tamaño del tablero (n reinas)
    - initial_individuals: lista de individuos (listas de tamaño n)
    """
    population = []

    # Validación básica
    if initial_individuals:
        for ind in initial_individuals:
            if len(ind) != n:
                raise ValueError(f"El individuo {ind} no tiene longitud {n}")
            if sorted(ind) != list(range(n)):
                raise ValueError(f"El individuo {ind} no es una permutación válida")

        population.extend(initial_individuals)

    # Completar con individuos aleatorios si hace falta
    while len(population) < size:
        individual = generate_individual(n)
        population.append(individual)

    return population

In [ ]:
# 2. Función de fitness
# Cálculo: Dos reinas se atacan en diagonal si
#   abs(col1 - col2) == abs(row1 - row2)

def fitness(individual):
    """Calcula el número de pares de reinas que NO se atacan en diagonal."""
    n = len(individual)
    non_attacking_pairs = 0

    for i in range(n):
        for j in range(i + 1, n):
            if abs(individual[i] - individual[j]) != abs(i - j):
                non_attacking_pairs += 1

    return non_attacking_pairs

In [332]:
# 3. Selección
# Método: selección elitista + ruleta (o aleatoria ponderada)
def selection(population, num_selected):
    """Selecciona los mejores individuos según el fitness."""
    sorted_population = sorted(population, key=fitness, reverse=True)
    return sorted_population[:num_selected]

In [333]:
# 4. Cruces (Crossover)
# Método: Order Crossover (OX)
def crossover(parent1, parent2):
    """Cruza dos padres para generar un hijo válido (sin filas repetidas)."""
    n = len(parent1)
    start, end = sorted(random.sample(range(n), 2))

    child = [None] * n
    # Copiamos un segmento del primer padre
    child[start:end] = parent1[start:end]

    # Rellenamos con genes del segundo padre sin duplicar
    fill_pos = end
    for gene in parent2:
        if gene not in child:
            if fill_pos >= n:
                fill_pos = 0
            while child[fill_pos] is not None:
                fill_pos += 1
                if fill_pos >= n:
                    fill_pos = 0
            child[fill_pos] = gene

    return child

In [334]:
# 5. Mutación
# Método: Permutación simple
def mutate(individual, mutation_rate):
    """Aplica mutación intercambiando dos genes con una cierta probabilidad."""
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(individual)), 2)
        individual[i], individual[j] = individual[j], individual[i]
    return individual

In [335]:
# 6. Generar nueva generación
# Se aplica: Selección de padres, Crossover, Mutación.
# Se repite hasta llenar la nueva generación
def create_new_generation(population, num_selected, mutation_rate, population_size):
    """Genera una nueva generación de individuos."""
    selected = selection(population, num_selected)
    new_population = selected.copy()

    while len(new_population) < population_size:
        parent1, parent2 = random.sample(selected, 2)
        child = crossover(parent1, parent2)
        child = mutate(child, mutation_rate)
        new_population.append(child)

    return new_population

In [336]:
# Función de Algoritmo Genético
def run_genetic_algorithm(n, population_size, max_generations, mutation_rate, selection_size_ratio=0.2, initial_individuals=None):
  TARGET_FITNESS = (n * (n - 1)) // 2
  
  # Generar población inicial
  population = (generate_population_with_seed(population_size, n, initial_individuals) 
      if initial_individuals is not None 
      else generate_population(population_size, n)
    )

  for generation in range(max_generations):
    # Ordenar por fitness
    population = sorted(population, key=fitness, reverse=True)

    # Mejor individuo actual
    best = population[0]
    best_fitness = fitness(best)

    print(f"Generación {generation}: Mejor fitness = {best_fitness}")

    # Verificamos si es solución perfecta
    if best_fitness == TARGET_FITNESS:
      print(f"\n¡Solución encontrada!: {best}")
      return best

    # Crear nueva generación
    num_selected = max(2, int(selection_size_ratio * population_size))
    population = create_new_generation(population, num_selected, mutation_rate, population_size)
  
  print("\nNo se encontró una solución válida.")
  return None

In [337]:
# Variables necesarias
N = 8                   # Número de reinas a colocar: Tamaño del tablero y cantidad de reinas a colocar.
POPULATION_SIZE = 100   # Tamaño de la población: Cantidad de soluciones candidatas en cada generación.
MAX_GENERATIONS = 10000000 # Límite superior de generaciones para detener el algoritmo si no se encuentra una solución perfecta.
MUTATION_RATE = 0.05    # Probabilidad de mutación para mantener diversidad genética.

In [338]:
# Probamos el algoritmo genético
solution = run_genetic_algorithm(N, POPULATION_SIZE, MAX_GENERATIONS, MUTATION_RATE)

Generación 0: Mejor fitness = 27
Generación 1: Mejor fitness = 27
Generación 2: Mejor fitness = 28

¡Solución encontrada!: [6, 2, 7, 1, 4, 0, 5, 3]


In [339]:
# Mostrar la solución en un tablero NxN
def view_solution(solution):
  print("Tablero (representación de filas por columna):")
  # Indices: 0 1 2 ...
  print("   ", end="")
  for i in range(len(solution)):
    print(f"{i}", end=" ")
  print("")

  for i in range(len(solution)):
    row = ['Q' if solution[i] == j else '.' for j in range(len(solution))]
    print(f" {i} ", end="")
    print(" ".join(row))
  
  print(f"Solución (genoma): {solution}")

view_solution(solution)

Tablero (representación de filas por columna):
   0 1 2 3 4 5 6 7 
 0 . . . . . . Q .
 1 . . Q . . . . .
 2 . . . . . . . Q
 3 . Q . . . . . .
 4 . . . . Q . . .
 5 Q . . . . . . .
 6 . . . . . Q . .
 7 . . . Q . . . .
Solución (genoma): [6, 2, 7, 1, 4, 0, 5, 3]


In [340]:
# Uno o más individuos iniciales dados manualmente
semillas = [
  [0, 4, 7, 5, 2, 6, 1, 3],
  [3, 1, 6, 2, 5, 7, 0, 4]
]

solution = run_genetic_algorithm(N, POPULATION_SIZE, MAX_GENERATIONS, MUTATION_RATE, 0.2, semillas)

view_solution(solution)

Generación 0: Mejor fitness = 28

¡Solución encontrada!: [0, 4, 7, 5, 2, 6, 1, 3]
Tablero (representación de filas por columna):
   0 1 2 3 4 5 6 7 
 0 Q . . . . . . .
 1 . . . . Q . . .
 2 . . . . . . . Q
 3 . . . . . Q . .
 4 . . Q . . . . .
 5 . . . . . . Q .
 6 . Q . . . . . .
 7 . . . Q . . . .
Solución (genoma): [0, 4, 7, 5, 2, 6, 1, 3]
